#### Setup

In [ ]:
from specific import *

### Specify the experiments to compare

In [ ]:
experiments = ("15_most_important", "no_temporal_shifts")
assert len(experiments) == 2

### Load data

In [ ]:
experiment_data = load_experiment_data(experiments)

### Predict BA

In [ ]:
n_threads = get_ncpus()

predictions = {}
errors = {}  # GFED4 - prediction.

for experiment, data in experiment_data.items():
    data["model"].n_jobs = n_threads
    with parallel_backend("threading", n_jobs=n_threads):
        pred = data["model"].predict(data["exog_data"])

    predictions[experiment] = get_masked_array(pred, data["master_mask"])
    errors[experiment] = get_masked_array(
        data["endog_data"] - pred, data["master_mask"]
    )

In [ ]:
diffs = predictions[experiments[1]] - predictions[experiments[0]]

In [ ]:
error_mag_diff = np.abs(errors[experiments[1]]) - np.abs(errors[experiments[0]])

In [ ]:
np.mean(error_mag_diff)

In [ ]:
plt.hist(get_unmasked(errors[experiments[0]]), bins=100)
plt.yscale("log")

In [ ]:
plt.hist(get_unmasked(errors[experiments[1]]), bins=100)
plt.yscale("log")

In [ ]:
plt.hist(get_unmasked(error_mag_diff), bins=100)
plt.yscale("log")

In [ ]:
model_name = f"comparison_{'_'.join(experiments)}"

# Plotting params.
figsize = (5.1, 2.8)
mpl.rcParams["figure.figsize"] = figsize
coast_linewidth = 0.3
date_str = "2010-01 to 2015-01"

In [ ]:
boundaries = [-1e-2, -1e-3, -1e-4, 0, 1e-4, 1e-3, 1e-2]

fig = cube_plotting(
    diffs,
    title=f"BA <No Temporal - Top 15>\n{date_str}",
    boundaries=boundaries,
    cmap="brewer_RdYlBu_11",
    cmap_midpoint=0,
    cmap_symmetric=True,
    colorbar_kwargs={"label": "Burned Area Fraction", "format": "%0.1e",},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(fig, f"ba_{model_name}", sub_directory="predictions")

In [ ]:
boundaries = [-1e-1, 0, 0.1 ** 0.5, 1, 1e1]

fig = cube_plotting(
    diffs / predictions[experiments[0]],
    title=f"BA <No Temporal - Top 15> / Top 15\n{date_str}",
    boundaries=boundaries,
    cmap="brewer_RdYlBu_11",
    cmap_midpoint=0,
    cmap_symmetric=False,
    colorbar_kwargs={"label": "1", "format": "%0.1e",},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(fig, f"rel_ba_{model_name}", sub_directory="predictions")

In [ ]:
vmax = np.max(plot_data)
boundaries = [1e-5, 1e-4, 1e-3, 1e-2, vmax]

fig = cube_plotting(
    np.abs(diffs),
    title=f"BA <|No Temporal - Top 15|>\n{date_str}",
    boundaries=boundaries,
    cmap="YlOrRd",
    colorbar_kwargs={"label": "Burned Area Fraction", "format": "%0.1e",},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(fig, f"ba_mean_abs_{model_name}", sub_directory="predictions")

In [ ]:
boundaries = [0.1, 0.316, 1, 3.16, 10]

fig = cube_plotting(
    np.abs(diffs) / predictions[experiments[0]],
    title=f"BA <|No Temporal - Top 15|> / Top 15\n{date_str}",
    boundaries=boundaries,
    cmap="YlOrBr",
    colorbar_kwargs={"label": "1", "format": "%0.1e",},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(
    fig, f"rel_ba_mean_abs_{model_name}", sub_directory="predictions"
)

#### Comparison of the error between the experiments

In [ ]:
# boundaries = [0.1, 0.316, 1, 3.16, 10]

for experiment, error in errors.items():
    fig = cube_plotting(
        np.abs(error),
        title=f"BA <|Error({experiment})|>\n{date_str}",
        #     boundaries=boundaries,
        cmap="YlOrBr",
        colorbar_kwargs={"label": "BA Fraction", "format": "%0.1e",},
        coastline_kwargs={"linewidth": coast_linewidth},
    )
    figure_saver.save_figure(
        fig, f"error_mag_{experiment}_{model_name}", sub_directory="predictions"
    )

In [ ]:
boundaries = [-5.3e-2, -1e-2, -1e-3, 0, 1e-3, 1e-2, 3.6e-2]

fig = cube_plotting(
    error_mag_diff,
    title=f"BA <|Error(No Temporal)| - |Error(Top 15)|>\n{date_str}",
    boundaries=boundaries,
    cmap="brewer_RdYlBu_11",
    cmap_midpoint=0,
    cmap_symmetric=False,
    colorbar_kwargs={"label": "BA Fraction", "format": "%0.1e",},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(
    fig, f"error_mag_diff_{model_name}", sub_directory="predictions"
)

In [ ]:
# boundaries = [0.1, 0.316, 1, 3.16, 10]

fig = cube_plotting(
    error_mag_diff
    / get_masked_array(
        experiment_data[experiments[0]]["endog_data"],
        experiment_data[experiments[0]]["master_mask"],
    ),
    title=f"BA <(|Error(No Temporal)| - |Error(Top 15)|) / GFED4>\n{date_str}",
    #     boundaries=boundaries,
    cmap="brewer_RdYlBu_11",
    cmap_midpoint=0,
    cmap_symmetric=False,
    colorbar_kwargs={"label": "1", "format": "%0.1e",},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(
    fig, f"rel_error_mag_diff_{model_name}", sub_directory="predictions"
)

### Load all feature data

In [ ]:
experiment = "all"
all_experiment_data = load_experiment_data((experiment,))["all"]

### Correlations between diffs and other variables - virtually no correlation between the diffs and any of the features

In [ ]:
diff_data = get_unmasked(diffs)
all_features = all_experiment_data["exog_data"].copy()
all_features["Diffs"] = diff_data

In [ ]:
with figure_saver("diff_corr_plot"):
    corr_plot(shorten_columns(all_features), fig_kwargs={"figsize": (12, 8)})

### Correlations between rel. diffs and other variables - virtually no correlation between the diffs and any of the features

In [ ]:
rel_diff_data = get_unmasked(diffs / predictions[experiments[0]])
rel_all_features = all_experiment_data["exog_data"].copy()
rel_all_features["Rel. Diffs"] = rel_diff_data

In [ ]:
with figure_saver("rel_diff_corr_plot"):
    corr_plot(shorten_columns(rel_all_features), fig_kwargs={"figsize": (12, 8)})

### Correlations between |diffs| and other variables - vritually no correlation between the diffs and any of the features

In [ ]:
abs_diff_data = np.abs(get_unmasked(diffs))
abs_all_features = all_experiment_data["exog_data"].copy()
abs_all_features["|Diffs|"] = abs_diff_data

In [ ]:
with figure_saver("abs_diff_corr_plot"):
    corr_plot(shorten_columns(abs_all_features), fig_kwargs={"figsize": (12, 8)})

### Correlations between rel. |diffs| and other variables - virtually no correlation between the diffs and any of the features

In [ ]:
rel_abs_diff_data = np.abs(get_unmasked(diffs / predictions[experiments[0]]))
rel_abs_all_features = all_experiment_data["exog_data"].copy()
rel_abs_all_features["Rel. |Diffs|"] = rel_abs_diff_data

In [ ]:
with figure_saver("rel_abs_diff_corr_plot"):
    corr_plot(shorten_columns(rel_abs_all_features), fig_kwargs={"figsize": (12, 8)})